In [1]:
import numpy as np
import cv2
from Utils import normalize, denormalize
from skimage.transform import resize
import imageio
import tensorflow as tf
import math

img_shape = (384,384,3)
downscale_factor = 4
h, w = img_shape[0], img_shape[1]
img_res = (h, w)        
low_res = (h // downscale_factor, w // downscale_factor) 

In [2]:
def process_img(img_path):
    img = imageio.imread(img_path).astype(np.float64)
    img_hr = resize(img,img_res)
    img_lr = resize(img,low_res)
    img_hr = normalize(np.array([img_hr]))
    img_lr = normalize(np.array([img_lr]))
    return (img_hr, img_lr)

def load_generator(path):
    Gen = tf.keras.models.load_model(path, compile=False)
    return Gen


In [3]:
def get_generated_images(generator, x_test_hr, x_test_lr):
    
    image_hr = denormalize(x_test_hr)
    gen_img = generator.predict(x_test_lr)
    generated_image = denormalize(gen_img)
    
    return (np.squeeze(image_hr), np.squeeze(generated_image))
    

In [4]:
def calc_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    
    return 20 * math.log10(255.0 / math.sqrt(mse))

In [5]:
def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calc_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if img1.ndim == 2:  #灰色 双通道
        return ssim(img1, img2)
    elif img1.ndim == 3: #彩色 三通道
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1[:,:,i], img2[:,:,i]))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:  #灰色 双通道
            return ssim(np.squeeze(img1), np.squeeze(img2))

In [6]:
def get_metrics(model_path, img_path):
    gen = load_generator(model_path)
    (img_hr,img_lr) = process_img(img_path)
    (image_hr, generated_image) = get_generated_images(gen, img_hr, img_lr)
##    tensorflow 集成有psnr与ssim，可直接调用
#     print("tf_psnr:",tf.image.psnr(image_hr,generated_image,255))
#     print("tf_ssim:",tf.image.ssim(image_hr,generated_image,255))
    psnr_ = calc_psnr(image_hr, generated_image)
    ssim_ = calc_ssim(image_hr,generated_image)
    return (psnr_, ssim_)

In [7]:
model_path = './Trained_model/gen_model3000.h5'
img_path = './datasets/DIV2K_train_HR/0001.png'
print(get_metrics(model_path, img_path))

tf_psnr: tf.Tensor(23.45739, shape=(), dtype=float32)
tf_ssim: tf.Tensor(0.53680414, shape=(), dtype=float32)
(384, 384, 3) 3
(23.457391255485504, 0.5368093380352544)
